In [ ]:
# Load the Patient & Cycle files
import pandas as pd
from pathlib import Path
import re

RAW_PATH = Path("../data/raw")
CLEAN_PATH = Path("../data/clean")

def extract_year(path):
    m = re.search(r"(20[0-2][0-9])", path.name)
    return int(m.group(1)) if m else None

patient_files = sorted(RAW_PATH.glob("*Patient_and_Cycle_Characteristics*.csv"))
patient_files


[WindowsPath('../data/raw/2020_Final_Assisted_Reproductive_Technology_(ART)_Patient_and_Cycle_Characteristics_20251127.csv'),
 WindowsPath('../data/raw/2021_Final_Assisted_Reproductive_Technology_(ART)_Patient_and_Cycle_Characteristics_20251127.csv'),
 WindowsPath('../data/raw/2022_Final_Assisted_Reproductive_Technology_(ART)_Patient_and_Cycle_Characteristics_20251127.csv')]

In [ ]:
# Load and combine patient datasets
patient_dfs = []

for f in patient_files:
    year = extract_year(f)
    print("Loading:", f.name)
    df = pd.read_csv(f)
    df["year"] = year
    patient_dfs.append(df)

patient_all = pd.concat(patient_dfs, ignore_index=True)
patient_all.shape


Loading: 2020_Final_Assisted_Reproductive_Technology_(ART)_Patient_and_Cycle_Characteristics_20251127.csv
Loading: 2021_Final_Assisted_Reproductive_Technology_(ART)_Patient_and_Cycle_Characteristics_20251127.csv
Loading: 2022_Final_Assisted_Reproductive_Technology_(ART)_Patient_and_Cycle_Characteristics_20251127.csv


(87232, 26)

In [3]:
patient_all.columns


Index(['Year', 'LocationAbbr', 'LocationDesc', 'FacilityName',
       'MedicalDirector', 'Address', 'City', 'ZipCode', 'Phone',
       'Clinic Status', 'Topic', 'Question', 'Breakout_Category', 'Breakout',
       'Data_Value', 'Data_Value_num', 'Data_Value_Footnote_Symbol',
       'Data_Value_Footnote', 'Cycle_Count', 'ClinicId', 'TopicId',
       'QuestionId', 'BreakOutCategoryId', 'BreakOutId', 'Geolocation',
       'year'],
      dtype='object')

In [ ]:
# Drop unnecessary columns
cols_to_drop = [
    "MedicalDirector",
    "Address",
    "ZipCode",
    "Phone",
    "Data_Value_Footnote_Symbol",
    "Data_Value_Footnote",
    "TopicId",
    "QuestionId",
    "BreakOutCategoryId",
    "BreakOutId"
]

patient_clean = patient_all.drop(columns=cols_to_drop, errors="ignore")
patient_clean.head()


,Year,LocationAbbr,LocationDesc,FacilityName,City,Clinic Status,Topic,Question,Breakout_Category,Breakout,Data_Value,Data_Value_num,Cycle_Count,ClinicId,Geolocation,year
0,2020,NaN,Other,National,NaN,Open,Patient & Cycle Characteristics,What percentage of transfers used a gestationa...,Age of Patient,35-37,3.7,3.7,"165,003","9,999",NaN,2020
1,2020,IA,Iowa,"Mid-Iowa Fertility, PC",Clive,Open,Patient & Cycle Characteristics,What were the reasons patients used ART?,Ovulatory dysfunction,Yes,18.2,18.2,862,37,POINT (-93.7840312 41.6024095),2020
2,2020,FL,Florida,The IVF Center,Winter Park,Reorganized,Patient & Cycle Characteristics,What were the reasons patients used ART?,Diminished ovarian reserve,Yes,38.3,38.3,287,295,POINT (-81.2756707 28.611928),2020
3,2020,IA,Iowa,"Mid-Iowa Fertility, PC",Clive,Open,Patient & Cycle Characteristics,What were the reasons patients used ART?,Other factor,Yes,5.6,5.6,862,37,POINT (-93.7840312 41.6024095),2020
4,2020,NY,New York,New York Fertility Institute,New York,Open,Patient & Cycle Characteristics,What were the reasons patients used ART?,Unexplained factor,Yes,24.0,24.0,104,227,POINT (-73.9615353 40.7793023),2020


In [5]:
patient_clean = patient_clean.dropna(subset=["Data_Value"])
patient_clean.shape


(87229, 16)

In [6]:
# Convert numeric-looking columns
patient_clean["Data_Value_num"] = pd.to_numeric(patient_clean["Data_Value_num"], errors="coerce")
patient_clean["Cycle_Count"] = pd.to_numeric(patient_clean["Cycle_Count"], errors="coerce")

patient_clean[["Data_Value", "Data_Value_num", "Cycle_Count"]].head()


,Data_Value,Data_Value_num,Cycle_Count
0,3.7,3.7,NaN
1,18.2,18.2,862.0
2,38.3,38.3,287.0
3,5.6,5.6,862.0
4,24.0,24.0,104.0


In [7]:
# Rename a couple of key columns to simpler names
patient_clean = patient_clean.rename(columns={
    "Data_Value": "data_value",
    "Data_Value_num": "data_value_num",
    "Cycle_Count": "cycle_count",
    "Clinic Status": "clinic_status"
})

# Make all column names lowercase_with_underscores
patient_clean.columns = (
    patient_clean.columns
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("__", "_")
)

patient_clean.head()
patient_clean.columns


Index(['year', 'locationabbr', 'locationdesc', 'facilityname', 'city',
       'clinic_status', 'topic', 'question', 'breakout_category', 'breakout',
       'data_value', 'data_value_num', 'cycle_count', 'clinicid',
       'geolocation', 'year'],
      dtype='object')

In [8]:
patient_clean.to_csv("../data/clean/patient_clean.csv", index=False)


In [9]:
patient_clean.shape


(87229, 16)